In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("Spark SQL Aleksei Borisikhin").getOrCreate()

In [3]:
spark

---

In [51]:
import re

import pyspark.sql.functions as f
from pyspark.sql.types import StringType, FloatType, ArrayType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.linalg import Vectors

In [52]:
target_courses = [
    [23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
    [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
    [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
    [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
    [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], 
    [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']
]

---

In [53]:
!hdfs dfs -head /labs/slaba02/DO_record_per_line.json

{"lang": "en", "name": "Accounting Cycle: The Foundation of Business Measurement and Reporting", "cat": "3/business_management|6/economics_finance", "provider": "Canvas Network", "id": 4, "desc": "This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appro

In [54]:
raw_data = spark.read.json('/labs/slaba02/DO_record_per_line.json')

In [55]:
raw_data.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
+--------------------+--------------------+---+----+--------------------+--------------+
only showing top 5 rows



In [56]:
raw_data.printSchema()

root
 |-- cat: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- id: long (nullable = true)
 |-- lang: string (nullable = true)
 |-- name: string (nullable = true)
 |-- provider: string (nullable = true)



---

In [57]:
def Tokenizer(x):
    regex = re.compile(u'[\w\d]{2,}', re.U)
    return regex.findall(x.lower())

tokenizer_udf = f.udf(Tokenizer, ArrayType(StringType()))

In [58]:
tf = HashingTF(inputCol='desc_words', outputCol='tf_feats', numFeatures=10000)
idf = IDF(inputCol='tf_feats', outputCol='tfidf_feats')

In [59]:
data = raw_data \
    .select('id', 'lang', 'desc') \
    .withColumn('desc_words', tokenizer_udf('desc'))
data = tf.transform(data)
data = idf.fit(data).transform(data).cache()

data.show(5)

+---+----+--------------------+--------------------+--------------------+--------------------+
| id|lang|                desc|          desc_words|            tf_feats|         tfidf_feats|
+---+----+--------------------+--------------------+--------------------+--------------------+
|  4|  en|This course intro...|[this, course, in...|(10000,[36,63,138...|(10000,[36,63,138...|
|  5|  en|This online cours...|[this, online, co...|(10000,[32,222,36...|(10000,[32,222,36...|
|  6|  fr|This course is ta...|[this, course, is...|(10000,[30,118,12...|(10000,[30,118,12...|
|  7|  en|We live in a digi...|[we, live, in, di...|(10000,[493,572,7...|(10000,[493,572,7...|
|  8|  en|This self-paced c...|[this, self, pace...|(10000,[32,115,13...|(10000,[32,115,13...|
+---+----+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [66]:
result = dict()

for course_info in target_courses:
    target_id = course_info[0]
    target_lang = course_info[1]
    
    cur_data = data \
        .filter(data['lang'] == target_lang) \
        .filter(data['id'] != target_id)
    
    target_tfidf = data \
        .filter(data['id'] == target_id) \
        .select(f.col('tfidf_feats'))
    target_tfidf = Vectors.dense(target_tfidf.take(1)[0]['tfidf_feats'])
    
    dist_func = f.udf(lambda x: float(x.dot(target_tfidf) / (target_tfidf.norm(2) * x.norm(2))), FloatType())
    
    course_res = cur_data \
        .withColumn('trg_dist', dist_func('tfidf_feats')) \
        .filter((~f.isnan('trg_dist')) & (~f.isnull('trg_dist'))) \
        .orderBy(f.col('trg_dist').desc(), f.col('name').asc(), f.col('id').asc()) \
        .select('id') \
        .collect()[:10]
    
    result[str(target_id)] = [cr.id for cr in course_res]

In [67]:
result

{'23126': [14760,
  13665,
  13782,
  20638,
  24419,
  15909,
  2724,
  25782,
  17499,
  13348],
 '21617': [21609,
  21616,
  21608,
  22298,
  21630,
  21628,
  21623,
  21508,
  21081,
  19417],
 '16627': [11431, 11575, 12247, 17964, 5687, 17961, 16694, 12660, 25010, 5558],
 '11556': [16488, 468, 13461, 23357, 19330, 7833, 9289, 10447, 22710, 11340],
 '16704': [1236, 1247, 1365, 1273, 20288, 1164, 8186, 1233, 8203, 875],
 '13702': [864, 21079, 8313, 1041, 28074, 8300, 1033, 13057, 21025, 1111]}

---

In [69]:
import json

with open('./../lab02.json', 'w') as res_file:
    json.dump(result, res_file)